In [1]:
import numpy as np
from sklearn.decomposition import NMF
import time
import copy
import math

In [2]:
X = np.array(np.random.choice([0,1],(100,20)))

In [3]:
X

array([[0, 1, 0, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 0, 1],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [5]:
#use the same init
n_factors=4
W0=np.random.random((X.shape[0],n_factors))
H0=np.random.random((n_factors,X.shape[1]))
print(W0.max(), W0.min(), W0.mean(), np.linalg.norm(W0))
print(H0.max(), H0.min(), H0.mean(), np.linalg.norm(H0))

0.9993326356321898 0.0047724633550728646 0.4744848605751693 11.10587651389246
0.9970518224489903 0.0010757902841025402 0.4363152034895211 4.696081830045622


In [21]:
%%time
n_factors=4
nmf = NMF(n_components=n_factors, init='custom',max_iter=500)
W = nmf.fit_transform(X,W=W0.astype(np.double),H=H0.astype(np.double)) #corresponding to the U matrix
H = nmf.components_ #corresponding to the V matrix
print(W.shape, H.shape)
print(W.max(), W.min(), W.mean(), np.linalg.norm(W))
print(H.max(), H.min(), H.mean(), np.linalg.norm(H))
print(nmf.get_params())

(100, 4) (4, 20)
0.8534930074730912 0.0 0.26878123064509496 6.868154390442655
2.457904908102121 0.0 0.48343643704125794 6.60419177612916
{'alpha': 0.0, 'beta_loss': 'frobenius', 'init': 'custom', 'l1_ratio': 0.0, 'max_iter': 500, 'n_components': 4, 'random_state': None, 'shuffle': False, 'solver': 'cd', 'tol': 0.0001, 'verbose': 0}
CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 16.5 ms


In [7]:
nmf.n_iter_

375

In [12]:
# manual decomposition
def updateU(X,U,V,n_factors,gamma=0.01):
    perm = np.array(range(n_factors))
    np.random.shuffle(perm)    
    for k in perm:
        for i in range(X.shape[0]):
#             grad=0
#             Z=0
#             for j in range(X.shape[1]):
#                 grad+=(X[i,j]-np.dot(U[i,:],V[:,j])+U[i,k]*V[k,j])*V[k,j]
#                 Z += V[k,j]*V[k,j] 
            grad =  np.sum(V[k,:]*(X[i,:]-np.dot(U[i,:],V[:])+(U[i,k]*V[k,:])))   
            Z = np.sum(np.square(V[k,:]))    
            U[i,k]=U[i,k]+gamma*(grad/Z-U[i,k])  #moving average    
    return U         

def updateV(X,U,V, n_factors,gamma=0.01):
    perm = np.array(range(n_factors))
    np.random.shuffle(perm)
    for k in perm:
        for j in range(X.shape[1]):
#             grad=0
#             Z=0
#             for i in range(X.shape[0]):
#                 grad += (X[i,j]-np.dot(U[i,:],V[:,j])+U[i,k]*V[k,j])*U[i,k]
#                 Z += U[i,k]*U[i,k]
            grad =  np.sum(U[:,k]*(X[:,j]-np.dot(U[:],V[:,j])+V[k,j]*U[:,k]))   
            Z = np.sum(np.square(U[:,k]))  
            V[k,j]=V[k,j]+gamma*(grad/Z-V[k,j])   #moving average  
            
    return V


In [13]:
def NMF1(X,U,V,n_factors,itr=500,tol=0.0001,gamma=0.1):
    i=0
    ut = math.inf
    vt = math.inf
    while (i < itr)and((ut>tol)or(vt>tol)):
        Uc = copy.deepcopy(U)
        Vc = copy.deepcopy(V)
        U = updateU(X, U, V, n_factors,gamma)
        V = updateV(X, U, V, n_factors,gamma)
        ut = np.linalg.norm(Uc-U)
        vt = np.linalg.norm(Vc-V)
        i+=1

    meta=[i,ut,vt]    
    return U, V, meta    


In [26]:
%%time
n_factors=4
U0 = copy.deepcopy(W0)
V0= copy.deepcopy(H0)
U, V, meta = NMF1(X,U0,V0, n_factors,itr=500,gamma=0.1)
print(U.max(), U.min(), U.mean(), np.linalg.norm(U))
print(V.max(), V.min(), V.mean(), np.linalg.norm(V)) 
print(meta)

1.4256359486225392 -0.7353469841280715 0.33773175031711955 9.711463203731865
1.0565875214032427 -0.28051829669813505 0.3539178638357291 4.361622729372063
[500, 0.004303717022975672, 0.0018612614344892034]
CPU times: user 4.17 s, sys: 0 ns, total: 4.17 s
Wall time: 4.17 s


In [27]:
np.linalg.norm(W-U)/(W.shape[0]*W.shape[1])

0.011676009680096841

In [28]:
np.linalg.norm(H-V)/(H.shape[0]*H.shape[1])

0.04298984278940894

In [29]:
np.abs(W-U).max()

0.7353469841280715

In [30]:
np.abs(H-V).max()

1.5553761178059644

In [31]:
np.abs(W-U).mean()

0.18365286237996753

In [32]:
np.abs(H-V).mean()

0.26195447960394697

Conclusion: The result is slightly different for the sklearn NMF and manual created NMF. It is because their optimization methods and regularization (thus also the loss function) are different, so the solutions they converge into can be a little different.